In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
import pymssql
from sqlalchemy import create_engine

In [ ]:
#Load Secret Environment variables
load_dotenv()

In [ ]:
# To run sql statements in jupyter cells
%load_ext sql

In [ ]:
# Setup MySQL connection
mysql_username = os.getenv('MYSQL_USERNAME')
password = os.getenv('MYSQL_PASSWORD')
mysql_host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
mysql_database = os.getenv('MYSQL_DATABASE')
mysql_connection_string = f"mysql+pymysql://{mysql_username}:{password}@{mysql_host}:{port}/{mysql_database}"
mysql_short = f"{mysql_username}@{mysql_host}"

%sql $mysql_connection_string

mysql_cnx = create_engine(mysql_connection_string)

# Reset variables
password = port = mysql_connection_string = None

In [ ]:
# Setup MSSQL connection

ms_username = os.getenv('MS_USERNAME')
password = os.getenv('MS_PASSWORD')
ms_host = os.getenv('MS_HOST')
port = os.getenv('MS_PORT')
database = os.getenv('MS_DATABASE')
ms_connection_string = f"mssql+pymssql://{ms_username}:{password}@{ms_host}:{port}/{database}"
ms_short = f"{ms_username}@{ms_host}"

ms_conn: pymssql.Connection = pymssql.connect(
    server=f"{ms_host}:{port}",
    user=ms_username,
    password=password,
    database=database
)


%sql $ms_connection_string

ms_cnx = create_engine(ms_connection_string)

ms_connect = ms_cnx.connect()

# Reset variables
password = port = database = ms_connection_string = None

In [ ]:
cursor = ms_conn.cursor()
for table_file in os.listdir('schemas'):
    with open(f'schemas/{table_file}', 'r') as f:
        table: str = table_file.split('.')[1]
        sql_lines = f.read()
        cursor.execute(sql_lines)
        # ms_connect.execute(sql_lines)
        ms_conn.commit()
    


In [ ]:
%%sql $ms_short
SELECT @@VERSION;

In [ ]:
%%sql $mysql_short activations <<
SELECT * 
FROM activations;

In [ ]:
tables = %sql $mysql_short SELECT table_name FROM information_schema.tables where table_schema = 'equitytest'
lst = []
for table in tables:
    lst.append(table)
newlst = [', '.join(map(str, x)) for x in lst]
# lst = [] 
print(newlst)
# newlst = []


In [ ]:
%%sql $ms_short
SELECT table_catalog[database], table_schema[schema], table_name[name], table_type[type]
FROM INFORMATION_SCHEMA.TABLES


In [ ]:
%%sql $ms_short 
SELECT * FROM activations;

In [ ]:
has_identity_sql

In [ ]:
ls = "activations"
has_identity_sql = f"SELECT OBJECTPROPERTY(OBJECT_ID('{ls}'), 'TableHasIdentity')"
%sql $ms_short has_identity << $has_identity_sql
has_identity


In [ ]:
%%sql $ms_short
SELECT OBJECTPROPERTY(OBJECT_ID('activations'), 'TableHasIdentity');


In [ ]:
has_identity[0][0]

In [ ]:
# print(len(newlst))
for ls in newlst:
    %sql $mysql_short source_data << SELECT * FROM {mysql_database}.$ls
    identity_insert: str = f"SET IDENTITY_INSERT [{ls}] ON;"
    has_identity_sql = f"SELECT OBJECTPROPERTY(OBJECT_ID('{ls}'), 'TableHasIdentity')"
    %sql $ms_short has_identity << $has_identity_sql
    if has_identity[0][0]:
        ms_connect.execute(identity_insert)
    else:
        print(f"{ls} has no identity")
    source_df: pd.DataFrame = source_data.DataFrame()
    source_df.replace('0000-00-00', '0001-01-01', inplace=True)
    # ms_conn.commit()
    print(f"""{ls} {source_df.to_sql(
        f"{ls}",
        con=ms_connect,
        if_exists="append",
        index=None
    )}""")
    identity_insert_off: str = f"SET IDENTITY_INSERT [{ls}] OFF;"
    # cursor.execute(identity_insert_off)
    if has_identity[0][0]:
        ms_connect.execute(identity_insert_off)
    # ms_conn.commit()


In [ ]:
%%sql $ms_short
SELECT * from [users]


In [ ]:
%sql $mysql_short source_data << SELECT * FROM activations


In [ ]:
for ls in newlst:
    %sql $mysql_short source_data << SELECT * FROM $ls
    source_df: pd.DataFrame = source_data.DataFrame()
    columns_list: list = list(source_df.columns)
    insertion_string = f"""
    INSERT INTO {ls}({str(columns_list)[1:-1]})
    """
    for i in range(len(source_data)):
        row_string: str = ""
        for column in columns_list:
            row_string += f"{source_df.loc[i, column]},"
        row_string = f"VAlUES({row_string})\n"
        insertion_string += row_string
    print(insertion_string)

In [ ]:
str(list(source_data.DataFrame().columns))[1:-1]

In [ ]:
source_data.DataFrame()

In [ ]:
for i in range(4):
    print(source_data.DataFrame().loc[i, "updated_at"])